In [31]:
#极坐标弧长曲线求解部分
from scipy import integrate
import sympy as sp
import numpy as np

In [32]:

def arc(t0,arc_length,distance_value) -> float:
    '''
    输入当前的角度t0和弧长arc_length
    返回前进到的角度t1
    积分是从t1到t0，因为t1的角度更小,前进过程中角度是在减小的
    '''
    c = distance_value / (2 * sp.pi) # 常数 c，用来计算螺线坐标
    a =  0.5 * c * (sp.sqrt(1 + t0**2) * t0 + sp.asinh(t0))
    t1 = sp.symbols('t1') # 定义符号变量t1,设定为未知数
    b =  0.5 * c * (sp.sqrt(1 + t1**2) * t1 + sp.asinh(t1))
    
    equation = a -b - arc_length
    #sp.pprint(equation)
    result = sp.nsolve(equation,t1, t0)
    
    return float(result)

'''
当前角度为32π，沿着弧线前进2cm
print(arc(32*np.pi,1))
'''

class Queue:
    '''
    实现一个队列，先入先出(FIFO)
    '''
    def __init__(self):
        self.queue = []
    
    def is_empty(self): 
        return len(self.queue) == 0
    
    def push(self, element):
        self.queue.append(element)
    
    def pop(self):
        if self.is_empty():
            return None
        else:
            return self.queue.pop(0)
    
    def get_size(self):
        return len(self.queue)

        
def search_vertex(head_theta,distance_value) -> list:
    c = distance_value / (2 * np.pi)  # 常数 c，用来计算螺线坐标
    theta_step = 0.0005     # 角度步长
    vertex = Queue()      # 队列，用于存储待处理的节点
    vertex_list = []      # 存储所有找到的顶点角度
    
    # 初始角度作为第一个顶点
    vertex.push(head_theta)
    vertex_list.append(head_theta)
    while not vertex.is_empty():
        if len(vertex_list) > 223: # 限制最多只能找到 224 个顶点
            #print("找完了",len(vertex_list))
            break
        
        # 从队列中取出当前的角度（作为线段的固定端点）
        current_theta = vertex.pop()
        #print(f"Processing theta: {current_theta}")
        
        # 当前点的坐标
        x0 = c * current_theta * np.cos(current_theta)
        y0 = c * current_theta * np.sin(current_theta)
        
        # 从当前 theta 开始，移动下一个点
        theta = current_theta
        while theta <= 32 * np.pi:
            # 尝试计算下一个点的角度
            next_theta = theta + theta_step
            x1 = c * next_theta * np.cos(next_theta)
            y1 = c * next_theta * np.sin(next_theta)
            
            # 计算两点之间的距离
            distance = np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
            if distance > 890:
                print(f"Warning!!!!! Distance too large: {distance}")
            
            # 处理距离为 286 的情况，且这是寻找“头部”的条件
            if distance >= 286 and len(vertex_list) == 1:
                #print("ok")
                #print(f"Found head: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 处理距离为 165 的情况
            if distance >= 165 and len(vertex_list) > 1 :
                #print(f"Found vertex at theta: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 更新 theta，继续向下搜索
            theta += theta_step
    
    return vertex_list
'''
# 测试函数
found_vertices = search_vertex(0.0)
print("Found vertices:", found_vertices)
'''
def distance(a,b,distance_value) -> float:
    '''
    输入两个角度a和b
    用近似替代方法计算两点之间的距离
    '''
    c = distance_value / (2 * np.pi)
    
    x0 = c * a * np.cos(a)
    y0 = c * a * np.sin(a)
    x1 = c * b * np.cos(b)
    y1 = c * b * np.sin(b)
    #print("x0,y0,x1,y1",x0,y0,x1,y1)
    return np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
#distance(32.67299999997398, 33.25299999997675)

def get_speed(t0,time,distance_value) -> list:
    '''
    输入初始角度和运动时间time
    确定时间微元dt为1e-7
    返回进入螺线的所有节点的坐标和速度
    '''
    dt = 1e-7
    
    current_arc = time*100 #当前头部的弧长
    updated_arc = (time+dt)*100 #dt时间内头部加上走过的弧长
    
    head_theta = arc(t0,current_arc,distance_value) #头部当前的角度
    updated_head_theta = arc(t0,updated_arc,distance_value) #dt后头部的角度
    
    current_theta_list = search_vertex(head_theta,distance_value)
    #print(current_theta_list)
    after_theta_list = search_vertex(updated_head_theta,distance_value)  

    
    speed_list = []
    for i in range(len(current_theta_list)):
        a = current_theta_list[i]
        b = after_theta_list[i]
        ds = distance(a,b,distance_value)
        speed = ds/dt
        speed_list.append(speed)
        
    return current_theta_list,speed_list
    
def crash_judge(thetaA,thetaE,thetaP,thetaQ,distance_value) -> bool:
    '''
    theta为某一节龙身的把手的角度，判断是否与龙头的把手相撞
    '''  
    c = distance_value / (2 * np.pi)
      
    xA = c * thetaA * np.cos(thetaA)
    yA = c * thetaA * np.sin(thetaA)
    xE = c * thetaE * np.cos(thetaE)
    yE = c * thetaE * np.sin(thetaE)
    
    k_AE = (yA - yE) / (xA - xE) 
    k_AE_vertical = -1 / k_AE
    
    d = yA - k_AE * xA
    b1 = d + 15*np.sqrt(1 + k_AE**2)
    b2 = d - 15*np.sqrt(1 + k_AE**2)
    
    c1 = yA - k_AE_vertical * xA +27.5*np.sqrt(1 + k_AE_vertical**2)
    c2 = yA - k_AE_vertical * xA -27.5*np.sqrt(1 + k_AE_vertical**2)
    
    xB1 = (c1 - b1) / (k_AE - k_AE_vertical)
    xB2 = (c1 - b2) / (k_AE - k_AE_vertical)
    xB3 = (c2 - b1) / (k_AE - k_AE_vertical)
    xB4 = (c2 - b2) / (k_AE - k_AE_vertical)
    
    yB1 = k_AE * xB1 + b1
    yB2 = k_AE * xB2 + b2
    yB3 = k_AE * xB3 + b1
    yB4 = k_AE * xB4 + b2
    
    xP = c * thetaP * np.cos(thetaP)
    yP = c * thetaP * np.sin(thetaP)
    xQ = c * thetaQ * np.cos(thetaQ)
    yQ = c * thetaQ * np.sin(thetaQ)
    
    k_PQ = (yP - yQ) / (xP - xQ)
    
    m = yP - k_PQ * xP
    
    #取PQ中点
    xS = (xP + xQ) / 2
    yS = (yP + yQ) / 2
    
    
    #对得到的四个点进行判断
    j1 = np.abs(k_PQ * xB1 - yB1 + m) / np.sqrt(k_PQ**2 + 1)
    j2 = np.abs(k_PQ * xB2 - yB2 + m) / np.sqrt(k_PQ**2 + 1)
    j3 = np.abs(k_PQ * xB3 - yB3 + m) / np.sqrt(k_PQ**2 + 1)
    j4 = np.abs(k_PQ * xB4 - yB4 + m) / np.sqrt(k_PQ**2 + 1)
    test_list = [j1,j2,j3,j4] #测试用
    d1 = np.sqrt((xS - xB1)**2 + (yS - yB1)**2)
    d2 = np.sqrt((xS - xB2)**2 + (yS - yB2)**2)
    d3 = np.sqrt((xS - xB3)**2 + (yS - yB3)**2)
    d4 = np.sqrt((xS - xB4)**2 + (yS - yB4)**2) 

    if j1 <= 15 and d1 <= 111 :
        return True

    if j2 <= 15 and d2 <= 111 :
        return True
    
    if j3 <= 15 and d3 <= 111 :
        return True
    
    if j4 <= 15 and d4 <= 111 :
        return True
    return False



In [33]:
import numpy as np

def main_q3(time,distance_value):
    theta_list , speed_list = get_speed(32*np.pi,time,distance_value)
    try:
        head_A = theta_list[0]
        head_B = theta_list[1]
        for i in range(2,len(theta_list)):
            body_P = theta_list[i]
            body_Q = theta_list[i+1]
            
            #print("searching time:",time)
            if crash_judge(head_A,head_B,body_P,body_Q,distance_value): 
                print(f"Crash!!!,Time At:{time}")
                print(f"current theta head_A:{head_A},head_B:{head_B},body_P:{body_P},body_Q:{body_Q}")
                return True , head_A
    except IndexError:
        return False , None
    
    return False , None
        
def get_distance(distance_value):    
    for i in np.arange(1,500,1):
        #print(main_q3(i,distance_value))
        condition , theta = main_q3(i,distance_value)
        if condition:
            print("OK")
            return theta
        
def search_distance():
    for i in np.arange(0,55,0.01):
        distance_value =  45 - i
        theta = get_distance(distance_value)
        distance_crash = (distance_value*theta)/(2*np.pi)
        print(f"current crash distance:{distance_crash},theta:{theta},distance_value:{distance_value}")
        if distance_crash >= 450:
            print(f"target distance:{distance_value},theta:{theta}")   
            
search_distance() 

Crash!!!,Time At:227
current theta head_A:61.3834861148588,head_B:62.042486114861944,body_P:67.41348611488759,body_Q:67.75648611488923
OK
current crash distance:439.6268357726625,theta:61.3834861148588,distance_value:45.0
Crash!!!,Time At:226
current theta head_A:61.59911808151674,head_B:62.25611808151988,body_P:67.61061808154544,body_Q:67.95261808154707
OK
current crash distance:441.0731479335355,theta:61.59911808151674,distance_value:44.99
Crash!!!,Time At:226
current theta head_A:61.58772709611636,head_B:62.2447270961195,body_P:67.60222709614507,body_Q:67.9442270961467
OK
current crash distance:440.89356422734824,theta:61.58772709611636,distance_value:44.98
Crash!!!,Time At:212
current theta head_A:64.67464540937411,head_B:65.29964540937709,body_P:70.75214540940313,body_Q:71.07914540940469
OK
current crash distance:462.8892292474966,theta:64.67464540937411,distance_value:44.97
target distance:44.97,theta:64.67464540937411


KeyboardInterrupt: 

# 初次搜索，从55到43，步长为1

Crash!!!,Time At:413
current theta head_A:25.91579464644424,head_B:27.239294646441156,body_P:31.788794646430553,body_Q:32.3847946464319
OK
current crash distance:226.85447521748435,theta:25.91579464644424,distance_value:55
Crash!!!,Time At:397
current theta head_A:29.4803665065802,head_B:30.652866506577467,body_P:35.37386650659044,body_Q:35.91936650659304
OK
current crash distance:253.3650868988814,theta:29.4803665065802,distance_value:54
Crash!!!,Time At:388
current theta head_A:30.135491511336333,head_B:31.30449151133361,body_P:36.02199151135119,body_Q:36.56749151135379
OK
current crash distance:254.19925913625053,theta:30.135491511336333,distance_value:53
Crash!!!,Time At:371
current theta head_A:33.792645957831986,head_B:34.847645957837024,body_P:39.72064595786029,body_Q:40.2246459578627
OK
current crash distance:279.6698655058525,theta:33.792645957831986,distance_value:52
Crash!!!,Time At:353
current theta head_A:37.54410898129619,head_B:38.50760898130079,body_P:43.51710898132471,body_Q:43.98610898132695
OK
current crash distance:304.7418569460597,theta:37.54410898129619,distance_value:51
Crash!!!,Time At:343
current theta head_A:38.560379070597406,head_B:39.51687907060197,body_P:44.50487907062579,body_Q:44.972879070628025
OK
current crash distance:306.8537468291421,theta:38.560379070597406,distance_value:50
Crash!!!,Time At:323
current theta head_A:42.705791621810484,head_B:43.583791621814676,body_P:48.67579162183899,body_Q:49.112291621841074
OK
current crash distance:333.04505392791583,theta:42.705791621810484,distance_value:49
Crash!!!,Time At:301
current theta head_A:47.191837589261986,head_B:48.00033758926585,body_P:53.166337589290514,body_Q:53.57383758929246
OK
current crash distance:360.519082844206,theta:47.191837589261986,distance_value:48
Crash!!!,Time At:278
current theta head_A:51.71329702881715,head_B:52.46479702882074,body_P:57.707797028845775,body_Q:58.091297028847606
OK
current crash distance:386.8300617486367,theta:51.71329702881715,distance_value:47
Crash!!!,Time At:254
current theta head_A:56.287057534668016,head_B:56.99155753467138,body_P:62.3155575346968,body_Q:62.678557534698534
OK
current crash distance:412.08471818205504,theta:56.287057534668016,distance_value:46
Crash!!!,Time At:227
current theta head_A:61.3834861148588,head_B:62.042486114861944,body_P:67.41348611488759,body_Q:67.75648611488923
OK
current crash distance:439.6268357726625,theta:61.3834861148588,distance_value:45
Crash!!!,Time At:181
current theta head_A:70.2672408882046,head_B:70.8547408882074,body_P:76.32574088823353,body_Q:76.635240888235
OK
current crash distance:492.06866389061497,theta:70.2672408882046,distance_value:44
target distance:44,theta:70.2672408882046
Crash!!!,Time At:131
current theta head_A:79.23606195235236,head_B:79.7680619523549,body_P:85.3405619523815,body_Q:85.62356195238286
OK
current crash distance:542.2648700266589,theta:79.23606195235236,distance_value:43
target distance:43,theta:79.23606195235236

# 第二次搜索，从45.0到44.9，步长为0.1
Crash!!!,Time At:227
current theta head_A:61.3834861148588,head_B:62.042486114861944,body_P:67.41348611488759,body_Q:67.75648611488923
OK
current crash distance:439.6268357726625,theta:61.3834861148588,distance_value:45.0
Crash!!!,Time At:211
current theta head_A:64.81943523944892,head_B:65.4444352394519,body_P:70.89343523947792,body_Q:71.21993523947948
OK
current crash distance:463.20337535256965,theta:64.81943523944892,distance_value:44.9
target distance:44.9,theta:64.81943523944892

# 第三次搜索，从45到44，步长为0.01
Crash!!!,Time At:227
current theta head_A:61.3834861148588,head_B:62.042486114861944,body_P:67.41348611488759,body_Q:67.75648611488923
OK
current crash distance:439.6268357726625,theta:61.3834861148588,distance_value:45.0
Crash!!!,Time At:226
current theta head_A:61.59911808151674,head_B:62.25611808151988,body_P:67.61061808154544,body_Q:67.95261808154707
OK
current crash distance:441.0731479335355,theta:61.59911808151674,distance_value:44.99
Crash!!!,Time At:226
current theta head_A:61.58772709611636,head_B:62.2447270961195,body_P:67.60222709614507,body_Q:67.9442270961467
OK
current crash distance:440.89356422734824,theta:61.58772709611636,distance_value:44.98
Crash!!!,Time At:212
current theta head_A:64.67464540937411,head_B:65.29964540937709,body_P:70.75214540940313,body_Q:71.07914540940469
OK
current crash distance:462.8892292474966,theta:64.67464540937411,distance_value:44.97
target distance:44.97,theta:64.67464540937411
# 得出最小的螺纹半径为 44.98
